In [10]:
import numpy as np
import random
from copy import deepcopy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import math,time,sys
from matplotlib import pyplot
import pandas as pd
from datetime import datetime
from functools import partial
import seaborn as sns 
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

MaxIter =5
pop_size = 20
omega = 0.99


def initialise(partCount, dim, trainX, testX, trainy, testy):    
    population=np.zeros((partCount,dim))
    minn = 1
    maxx = math.floor(0.5*dim)
    
    if maxx<minn:
        maxx = minn + 1
        #not(c[i].all())
    
    for i in range(partCount):
        random.seed(i**3 + 10 + time.time() ) 
        no = random.randint(minn,maxx)
        if no == 0:
            no = 1
        random.seed(time.time()+ 100)
        pos = random.sample(range(0,dim-1),no)
        for j in pos:
            population[i][j]=1
            
    return population

def fitness(agent, trainX, testX, trainy, testy):
    # print(agent)
    cols=np.flatnonzero(agent)
    # print(cols)
    val=1
    if np.shape(cols)[0]==0:
        return val    
    clf=KNeighborsClassifier(n_neighbors=5)
    #clf=MLPClassifier(alpha=0.001, hidden_layer_sizes=(1000,500,100),max_iter=2000,random_state=4)
    train_data=trainX[:,cols]
    test_data=testX[:,cols]
    clf.fit(train_data,trainy)
    val=1-clf.score(test_data,testy)

    #in case of multi objective  []
    set_cnt=sum(agent)
    set_cnt=set_cnt/np.shape(agent)[0]
    val=omega*val+(1-omega)*set_cnt
    return val

def test_accuracy(agent, trainX, testX, trainy, testy):
    cols=np.flatnonzero(agent)
    val=1
    if np.shape(cols)[0]==0:
        return val    
    # clf = RandomForestClassifier(n_estimators=300)
    #clf=MLPClassifier(alpha=0.001, hidden_layer_sizes=(1000,500,100),max_iter=2000,random_state=4)
    clf=KNeighborsClassifier(n_neighbors=5)
    # clf=MLPClassifier( alpha=0.01, max_iterno=1000) #hidden_layer_sizes=(1000,500,100)
    #cross=4
    #test_size=(1/cross)
    #X_train, X_test, y_train, y_test = train_test_split(trainX, trainy,  stratify=trainy,test_size=test_size)
    train_data=trainX[:,cols]
    test_data=testX[:,cols]
    clf.fit(train_data,trainy)
    val=clf.score(test_data,testy)
    return val

def onecnt(agent):
    return sum(agent)

def sigmoid(gamma):
    if gamma < 0:
        return 1 - 1/(1 + math.exp(gamma))
    else:
        return 1/(1 + math.exp(-gamma))
    
def WOA(dataset):
    df = pd.read_csv(dataset)
    a, b = np.shape(df)
    data = df.values[:,0:b-1]
    label = df.values[:,b-1]
    dimension = data.shape[1]
    
    cross = 5
    test_size = (1/cross)
    trainX, testX, trainy, testy = train_test_split(data, label,stratify=label ,test_size=test_size,random_state=(7+17*int(time.time()%1000)))
    clf=KNeighborsClassifier(n_neighbors=5)
    clf.fit(trainX,trainy)
    val=clf.score(testX,testy)
    whole_accuracy = val
    print("Total Acc: ",val)
    
    pop = initialise(pop_size, dimension, trainX, testX, trainy, testy)
    fit = []
    for i in range(pop_size):
        fit.append(fitness(pop[i], trainX, testX, trainy, testy))
    ind = np.argsort(fit)
    gbest = pop[ind[0]].copy()
    gbest_fit = fit[ind[0]].copy()
    
    for n in range(MaxIter):
        a = 2 - 2 * n / (MaxIter - 1)            # linearly decreased from 2 to 0

        for j in range(pop_size):

            r = np.random.rand()
            A = 2 * a * r - a
            C = 2 * r
            l = np.random.uniform(-1, 1)
            p = np.random.rand()
            b = 1

            if (p < 0.5) :
                if np.abs(A) < 1:
                    D = np.abs(C * gbest - pop[j] )
                    pop[j] = gbest - A * D
                else :
                    x_rand = pop[np.random.randint(pop_size)] 
                    D = np.abs(C * x_rand - pop[j])
                    pop[j] = (x_rand - A * D)
            else:
                D1 = np.abs(gbest - pop[j])
                pop[j] = D1 * np.exp(b * l) * np.cos(2 * np.pi * l) + gbest
                
                
        for i in range(pop_size):
            for j in range(dimension):
                if (sigmoid(pop[i][j]) > random.random()):
                    pop[i][j] = 1
                else:
                    pop[i][j] = 0
                    
    ind = np.argsort(fit)
    bestpop = pop[ind[0]].copy()
    bestfit = fit[ind[0]].copy()
    
    testAcc = test_accuracy(bestpop, trainX, testX, trainy, testy)
    featCnt = onecnt(bestpop)
    #print("best agent: ", bestpop)
    print("Test Accuracy: ", testAcc)
    print("#Features: ", featCnt)
            
    return testAcc, featCnt, bestpop

In [11]:
%%time
Ta,fc,bp = WOA('VoxForge512.csv')

Total Acc:  0.9055555555555556
Test Accuracy:  0.9277777777777778
#Features:  104.0
CPU times: user 381 ms, sys: 15.3 ms, total: 397 ms
Wall time: 396 ms


In [12]:
List = []

for i in range(0,len(bp)):
    if bp[i] == 1:
        List.append(i)
        
df_train = pd.read_csv("VoxForge512.csv")
y = (df_train['label'])
df_train1 = df_train[df_train.columns[List]]
l1 = []
for i in range(0,len(List)):
    l1.append(i)
    
df_train1.columns = l1

df_train1['label'] = y
df_train1

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,0,1,2,3,4,5,6,7,8,9,...,95,96,97,98,99,100,101,102,103,label
0,0.003680,0.003032,0.002665,0.001803,0.000742,0.000436,0.001257,0.000705,0.001716,0.000715,...,5.710641,14.609205,2.477992,2.826686,17.031933,11.462402,0.031459,0.009660,0.017622,English
1,0.002047,0.001964,0.002996,0.000613,0.000193,0.000170,0.000274,0.000303,0.000288,0.000407,...,2.949254,9.241147,1.788440,4.047075,17.947880,11.421647,0.029206,0.013936,0.024158,English
2,0.001723,0.001354,0.001195,0.000786,0.000294,0.000363,0.000756,0.000745,0.000763,0.001803,...,3.070010,10.038836,1.958939,4.674224,20.371273,11.088083,0.029121,0.015527,0.028974,English
3,0.010772,0.011722,0.008530,0.003547,0.002010,0.001000,0.001169,0.001460,0.000680,0.000531,...,0.105738,0.475082,0.109862,0.421951,1.997327,9.626831,0.023270,0.010665,0.016978,English
4,0.004742,0.004911,0.003138,0.002107,0.001037,0.000500,0.000427,0.000688,0.000511,0.000421,...,1.887431,5.555333,0.912959,0.978619,5.984027,9.285885,0.022074,0.008326,0.017641,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,0.000790,0.001097,0.000726,0.000450,0.000327,0.000301,0.001660,0.001563,0.000943,0.002466,...,0.010332,0.049385,0.008898,0.007885,0.001688,29.929813,0.064538,0.025733,0.011313,Spanish
896,0.000087,0.000120,0.000092,0.000099,0.000090,0.000129,0.000310,0.000199,0.000323,0.000344,...,0.004692,0.013272,0.002336,0.003356,0.001586,9.986907,0.010794,0.003350,0.002254,Spanish
897,0.000108,0.000099,0.000042,0.000035,0.000028,0.000037,0.000082,0.000123,0.000086,0.000190,...,0.001028,0.006380,0.001357,0.001960,0.000688,9.939436,0.012853,0.005161,0.003221,Spanish
898,0.000010,0.000007,0.000008,0.000007,0.000017,0.000008,0.000012,0.000017,0.000008,0.000023,...,0.001149,0.002084,0.000473,0.001001,0.000519,9.965343,0.008469,0.003256,0.002046,Spanish


In [13]:
df_train1.to_csv('VF_BWOA.csv')